# 기계학습기초 10주차 과제

# 데이터사이언스전공 201804217 유제우


## Logistic 회귀를 이용한 이진 분류

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 사용할 데이터 불러오기

fish = pd.read_csv('https://bit.ly/fish_csv')
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

In [3]:
# 학습데이터, 테스트데이터 나누기

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state = 42
)

In [4]:
train_input[:3]

array([[720.    ,  35.    ,  40.6   ,  16.3618,   6.09  ],
       [500.    ,  45.    ,  48.    ,   6.96  ,   4.896 ],
       [  7.5   ,  10.5   ,  11.6   ,   1.972 ,   1.16  ]])

In [5]:
#특성 정규화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [6]:
# 이진분류를 위해 데이터 골라내기: Bream과 Smelt만 분류해보자
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [7]:
#
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)

LogisticRegression()

In [8]:
# decision_function 값
dcsn = lr.decision_function(train_bream_smelt[:5])
print(dcsn)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [9]:
# 각 클래스에 속할 확률
proba = lr.predict_proba(train_bream_smelt[:5])
print(proba)

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


----

제우's 독학

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_bream_smelt, target_bream_smelt)

DecisionTreeClassifier()

In [12]:
dcsn = dt.decision_function(train_bream_smelt[:5])
print(dcsn)

AttributeError: ignored

---------------------

1. 위 코드에서 dcsn과 proba의 관계는?

  -> dcsn은 decision_function으로 z값을 나타낸다. proba는 decision_finction에서 나온 z값을 시그모이드함수를 사용하여 확률로 바꾼값이다. 이것으로 Bream과 Smelt의 확률을 예측할 수 있고, 이 둘 확률의 합은 1이 된다.

-----------------

2. predict_proba함수를 사용하지 않고 proba를 얻을 수 있는 코드를 작성하시오.
(Hint: expit와 column_stack을 이용하여 작성할 수 있다.)

In [ ]:
from scipy.special import expit

A = 1-expit(dcsn)
B = expit(dcsn)

In [ ]:
import numpy as np 

A_B = np.column_stack((B, A))
print(A_B)

[[0.00240145 0.99759855]
 [0.97264817 0.02735183]
 [0.00513928 0.99486072]
 [0.01415798 0.98584202]
 [0.00232731 0.99767269]]


----
교수님 답

In [13]:
z = dcsn

from scipy.special import expit

proba_ = np.column_stack([1-expit(z), expit(z)])

print(proba_)

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


# Logistic회귀를 이용한 다중 분류

In [ ]:
# 다중분류를 위한 모델과 학습
lr = LogisticRegression(C = 20, max_iter = 1000)
lr.fit(train_scaled, train_target)

LogisticRegression(C=20, max_iter=1000)

In [ ]:
# 예측
print(lr.predict(test_scaled[:3]))

['Perch' 'Smelt' 'Pike']


In [ ]:
# 테스트 데이터 3개가 각 클래스에 속할 확률(predict_proba)
proba = lr.predict_proba(test_scaled[:3])
print(np.round(proba, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]


In [ ]:
# decision_function 값들
decision = lr.decision_function(test_scaled[:3])
print(np.round(decision, decimals = 2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]]


In [ ]:
# softmax 함수를 이용한 predict_proba 계산

from scipy.special import softmax
proba_sm = softmax(decision, axis = 1)
print(np.round(proba_sm, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]


---------------------------------

3. proba와 proba_sm . 위 코드를 실행시켜 와 값이 일치함을 확인하시오.

  -> 일치한다.

In [ ]:
# 테스트 데이터 3개가 각 클래스에 속할 확률(predict_proba)
proba = lr.predict_proba(test_scaled[:3])
print(np.round(proba, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]


In [ ]:
# softmax 함수를 이용한 predict_proba 계산

from scipy.special import softmax
proba_sm = softmax(decision, axis = 1)
print(np.round(proba_sm, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]


----------------------------

4. p.191 softmax proba_sm . 다음의 교제 정의를 이용하여 을 계산하는 코드를 작성하시오


s1 = e^z / e_sum

In [ ]:
np.exp(decision)

array([[1.50628887e-03, 2.80732264e+00, 1.74797177e+02, 6.53061904e-02,
        2.81874117e+01, 1.38613638e+00, 5.30211910e-01],
       [1.92215778e-05, 6.87015022e+00, 1.18037462e+02, 9.08555949e-02,
        1.96503513e+01, 2.54361873e+03, 1.41270298e-02],
       [1.30980810e-02, 1.96340260e-03, 2.39138358e+01, 6.56359237e+02,
        1.05653451e+01, 1.12590065e+01, 2.08261200e-02]])

In [ ]:
dcsn = np.exp(decision)
length = len(dcsn[0])

In [ ]:
# s = e^z / e_sum

s = [[0] * length, [0] * length, [0] * length]
esum = 0
for i in range(len(dcsn)):
  for j in range(length):
    esum += dcsn[i][j]
  for k in range(length):
    s[i][k] = dcsn[i][k] / esum
s

[[7.2496130268749e-06,
  0.013511354392802457,
  0.8412807914588042,
  0.00031431196050320156,
  0.13566310584825494,
  0.006671331476271952,
  0.0025518552503363046],
 [6.637155041740168e-09,
  0.0023722429377817373,
  0.04075799312590172,
  3.1372173292523706e-05,
  0.00678520927637318,
  0.8783041681483829,
  4.878022402608563e-06],
 [3.6401859585973433e-06,
  5.456639462891248e-07,
  0.0066460735090112764,
  0.18241371960764483,
  0.0029362943090786085,
  0.0031290749720760823,
  5.7879432392024594e-06]]

In [ ]:
result = np.round(s, decimals=3)
print(result)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.002 0.041 0.    0.007 0.878 0.   ]
 [0.    0.    0.007 0.182 0.003 0.003 0.   ]]


----------------------

In [ ]:
z = decision

In [ ]:
z

array([[ -6.49810635,   1.03223123,   5.16362632,  -2.72866845,
          3.33887548,   0.32652029,  -0.63447852],
       [-10.85947707,   1.92718597,   4.77100205,  -2.3984839 ,
          2.97809521,   7.84134304,  -4.25966531],
       [ -4.33528955,  -6.23307629,   3.17445719,   6.48670826,
          2.35757931,   2.42116839,  -3.87154731]])

In [ ]:
esum = np.sum(np.exp(z), axis = 1)

In [ ]:
esum

array([ 207.77507248, 2688.28169779,  702.13331168])

In [ ]:
ESUM = np.repeat(esum.reshape(z.shape[0], -1), z.shape[1], axis = 1)

In [ ]:
print(np.round(np.exp(z)/ESUM, decimals = 3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]]
